LOAD THE MODEL AND MARK LABELS ON THE FACE

In [ ]:
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import time

# Load the saved model
from keras.models import load_model
model = load_model('../model/model1.h5')  # <-- Saved model path


def detect_points(face_img):
    me  = np.array(face_img)/255
    x_test = np.expand_dims(me, axis=0)
    x_test = np.expand_dims(x_test, axis=3)

    y_test = model.predict(x_test)
    label_points = (np.squeeze(y_test)*48)+48 
    
    return label_points
    
# Load haarcascade
face_cascade = cv2.CascadeClassifier('../conf/opencv_haarcascade/haarcascade_frontalface_default.xml')
dimensions = (96, 96)

# Enter the path to your test image
img = cv2.imread('../raw_images/Test_Image_1.JPG')

default_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)
# faces = face_cascade.detectMultiScale(gray_img, 4, 6)

faces_img = np.copy(gray_img)

plt.rcParams["axes.grid"] = False


all_x_cords = []
all_y_cords = []

for i, (x,y,w,h) in enumerate(faces):
    
    h += 10
    w += 10
    x -= 5
    y -= 5
    
    just_face = cv2.resize(gray_img[y:y+h,x:x+w], dimensions)
    cv2.rectangle(faces_img,(x,y),(x+w,y+h),(255,0,0),1)
    
    scale_val_x = w/96
    scale_val_y = h/96
    
    label_point = detect_points(just_face)
    all_x_cords.append((label_point[::2]*scale_val_x)+x)
    all_y_cords.append((label_point[1::2]*scale_val_y)+y)
   
   
    plt.imshow(just_face, cmap='gray')
    plt.plot(label_point[::2], label_point[1::2], 'ro', markersize=5)
    plt.show()
    
    
plt.imshow(default_img)    
plt.plot(all_x_cords, all_y_cords, 'wo',  markersize=3)
plt.show()

MASK EYES AND MOUTH

In [ ]:
rt_eye_pts = np.array([[all_x_cords[0][2],all_y_cords[0][2]+10],[all_x_cords[0][0],all_y_cords[0][0]+10],[all_x_cords[0][3],all_y_cords[0][3]+10],[all_x_cords[0][7],all_y_cords[0][7]+10],[all_x_cords[0][6],all_y_cords[0][6]+10]], np.int32)
lt_eye_pts = np.array([[all_x_cords[0][1],all_y_cords[0][1]+10],[all_x_cords[0][5],all_y_cords[0][5]+10],[all_x_cords[0][9],all_y_cords[0][9]+10],[all_x_cords[0][8],all_y_cords[0][8]+10],[all_x_cords[0][4],all_y_cords[0][4]+10]], np.int32)
mth_pts = np.array([[all_x_cords[0][10],all_y_cords[0][10]-10],[all_x_cords[0][12],all_y_cords[0][12]-10],[all_x_cords[0][11],all_y_cords[0][11]-10],[all_x_cords[0][13],all_y_cords[0][13]-10]], np.int32)
masked_img = cv2.fillPoly(default_img, [rt_eye_pts], 255)
masked_img = cv2.fillPoly(masked_img, [lt_eye_pts], 255)
masked_img = cv2.fillPoly(masked_img, [mth_pts], 255)
plt.imshow(masked_img)